In [1]:
# Hyperparameters searching for a deep feedforward neural network
# Using Keras to implement a neural network and search its hyperparameters for handwritten digits classification.

import keras
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import StandardScaler
import numpy as np


# Load data and data standardization
def load_data():
    '''Load the MNIST dataset'''
    
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    return (X_train, y_train, X_test, y_test)

def data_std(X_train, X_test):
    '''Data standardization
    
    Parameters
    ----------
    X_train: origianl training set
    X_test: original test set
    
    Returns
    -------
    X_train_std: rescaled training set
    X_test_std: rescaled test set
    '''
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    
    return X_train_std, X_test_std


def create_NN(n_features = 784, n_outputs = 10):
    '''create a deep feedforwad neural network using keras
    
    Parameters
    -----------
    n_features: the number of input features/units
    n_output: the number of output units
    
    Returns
    -------
    myNN: the neural network model
    
    '''
    myNN = keras.models.Sequential()

    ## Hidden Layer
    myNN.add(keras.layers.Dense(
    units=100,
    input_dim=n_features,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='sigmoid'))
    
    ## Output Layer
    myNN.add(keras.layers.Dense(
    units=n_outputs,
    input_dim= 100,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='softmax'))

    # Compile model
    sgd_optimizer = keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=.9)
    myNN.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy')
    
    return myNN

def nn_params_search(nn, X_train, y_train):
    '''Search best paramaters for svm classifier
    
    Parameters
    ----------
    X_train: features
    y_train: target of the input

    
    Returns
    -------
    best_params_
    
    Example grid: (you can customize the search graid by youself)
    param_grid = [{'batch_size': [64, 128], 'epochs' : [10, 30, 50]}]
        
    '''
    param_grid = [{'batch_size': [64, 128], 'epochs' : [10, 30, 50]}]
    gs_cv = GridSearchCV(estimator = nn, cv = 3, param_grid=param_grid , scoring = 'accuracy', verbose = 0)

    # model training
    gs_cv.fit(X_train, y_train)
    
    return gs_cv.best_params_


def retrain_best_nn(best_params, X_train, y_train):
    '''
    Retrain a svm classifier using the best parameters
    
    Paramters
    ----------
    best_params:
    X_train: data input of the training set
    y_train: target of the input
    
    Returns
    ---------
    bestNN: the nn classifier trained using the best parameters
    
    '''
    bestNN = create_NN(X_train.shape[1], y_train.shape[1])
    bestNN.summary()
    bestNN.fit(X_train, y_train,
                    batch_size = best_params['batch_size'], epochs = best_params['epochs'],
                    verbose = 1,
                    validation_split=0.1)
    
    return bestNN


def performance_acc(y, y_pred):
    ''' calculate the concusion matrix and average accuracy
    
        Parameters
        ----------
        y: real target
        y_pred: prediction
        
        Returns
        -------
        cm: confusion matrix
        acc: accuracy
    '''
    correct_preds = np.sum(y == y_pred, axis=0)
    acc = correct_preds / y.shape[0]
    
    cm = confusion_matrix(y, y_pred)
    
    return cm, acc


from keras.wrappers.scikit_learn import KerasClassifier

if __name__ == '__main__':
    
    (X_train, y_train, X_test, y_test) = load_data()
    
    # Reshaping the training and test sets to N * 784.
    X_train_1 =  np.reshape(X_train, [X_train.shape[0], X_train.shape[1] * X_train.shape[2]])
    X_test_1 = np.reshape(X_test, [X_test.shape[0], X_test.shape[1] * X_test.shape[2]])
    print('1. X_train: {}, X_train_1: {}'.format(X_train.shape, X_train_1.shape))
    ##
    
    # Transforming y_train to one-hot vectors using keras.utils.to_categorical.
    y_train_onehot = keras.utils.to_categorical(y_train)
    print('y_train_onehot: {}'.format(y_train_onehot.shape))
    ##
    
    # Creating a deep feedforward neural network
    myNN = create_NN(X_train_1.shape[1], y_train_onehot.shape[1])
    myNN.summary()
    myNN1 = KerasClassifier(build_fn = create_NN, batch_size = 64, epochs = 50)
    
    # Searching best paprameters, and report the performance
    best_params = nn_params_search(myNN1, X_train_1, y_train)
    print('Best parameters: ', best_params)
    
    bestNN = retrain_best_nn(best_params, X_train_1, y_train_onehot)
    y_test_pred = bestNN.predict_classes(X_test_1)
    cm, acc = performance_acc(y_test, y_test_pred)
    print('Confusion matrix:\n', cm)
    print('Accuracy =    {:.3f}%'.format(acc*100))
    
    # Searching best nn parameters after data standardization, and report the performance
    X_train_std, X_test_std = data_std(X_train_1, X_test_1)
    
    best_params =  nn_params_search(myNN1, X_train_std, y_train)
    print('Best parameters: ', best_params)
    
    bestNN_std = retrain_best_nn(best_params, X_train_std, y_train_onehot)
    y_test_std_pred = bestNN_std.predict_classes(X_test_std)
    cm1, acc1 = performance_acc(y_test, y_test_std_pred)
    print('Confusion matrix:\n', cm1)
    print('Accuracy =    {:.3f}%'.format(acc1*100))




Using TensorFlow backend.


1. X_train: (60000, 28, 28), X_train_1: (60000, 784)
y_train_onehot: (60000, 10)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
40000/40000 [==============================] - 3s 64us/step - loss: 0.9895
Epoch 2/10
40000/40000 [==============================] - 2s 51us/step - loss: 0.5029
Epoch 3/10
40000/40000 [==============================] - 2s 56us/step - loss: 0.4092
Epoch 4/10
40000/40000 [==============================] - 2s 60us/step - loss: 0.3609
Epoch 5/10
40000/40000 [==============================] - 2s 60us/step - loss: 0.3280
E

40000/40000 [==============================] - 1s 21us/step - loss: 0.1647
Epoch 28/30
40000/40000 [==============================] - 1s 20us/step - loss: 0.1654
Epoch 29/30
40000/40000 [==============================] - 1s 21us/step - loss: 0.1599
Epoch 30/30
40000/40000 [==============================] - 1s 30us/step - loss: 0.1632
Epoch 1/30
40000/40000 [==============================] - 1s 20us/step - loss: 1.0083
Epoch 2/30
40000/40000 [==============================] - 1s 37us/step - loss: 0.5064
Epoch 3/30
40000/40000 [==============================] - 2s 47us/step - loss: 0.4100
Epoch 4/30
40000/40000 [==============================] - 2s 46us/step - loss: 0.3591
Epoch 5/30
40000/40000 [==============================] - 2s 49us/step - loss: 0.3269
Epoch 6/30
40000/40000 [==============================] - 2s 54us/step - loss: 0.3054
Epoch 7/30
40000/40000 [==============================] - 2s 49us/step - loss: 0.2890
Epoch 8/30
40000/40000 [==============================] - 2s 4

40000/40000 [==============================] - 2s 44us/step - loss: 0.2373
Epoch 12/50
40000/40000 [==============================] - 2s 50us/step - loss: 0.2288
Epoch 13/50
40000/40000 [==============================] - 2s 60us/step - loss: 0.2208
Epoch 14/50
40000/40000 [==============================] - 2s 45us/step - loss: 0.2151
Epoch 15/50
40000/40000 [==============================] - 2s 46us/step - loss: 0.2075
Epoch 16/50
40000/40000 [==============================] - 2s 54us/step - loss: 0.2057
Epoch 17/50
40000/40000 [==============================] - 2s 45us/step - loss: 0.1997
Epoch 18/50
40000/40000 [==============================] - 2s 52us/step - loss: 0.1917
Epoch 19/50
40000/40000 [==============================] - 2s 42us/step - loss: 0.1917
Epoch 20/50
40000/40000 [==============================] - 1s 32us/step - loss: 0.1853
Epoch 21/50
40000/40000 [==============================] - 2s 38us/step - loss: 0.1840
Epoch 22/50
40000/40000 [==============================

40000/40000 [==============================] - 1s 23us/step - loss: 0.4062
Epoch 6/10
40000/40000 [==============================] - 1s 22us/step - loss: 0.3712
Epoch 7/10
40000/40000 [==============================] - 1s 25us/step - loss: 0.3495
Epoch 8/10
40000/40000 [==============================] - 1s 23us/step - loss: 0.3272
Epoch 9/10
40000/40000 [==============================] - 1s 21us/step - loss: 0.3108
Epoch 10/10
40000/40000 [==============================] - 1s 22us/step - loss: 0.3008
Epoch 1/10
40000/40000 [==============================] - 1s 21us/step - loss: 1.2184
Epoch 2/10
40000/40000 [==============================] - 1s 18us/step - loss: 0.6210
Epoch 3/10
40000/40000 [==============================] - 1s 21us/step - loss: 0.4885
Epoch 4/10
40000/40000 [==============================] - 1s 18us/step - loss: 0.4217
Epoch 5/10
40000/40000 [==============================] - 1s 19us/step - loss: 0.3796
Epoch 6/10
40000/40000 [==============================] - 1s 21u

...

40000/40000 [==============================] - 1s 26us/step - loss: 0.1809
Epoch 49/50
40000/40000 [==============================] - 1s 31us/step - loss: 0.1792
Epoch 50/50
40000/40000 [==============================] - 1s 20us/step - loss: 0.1775
Epoch 1/10
40000/40000 [==============================] - 1s 15us/step - loss: 1.7067
Epoch 2/10
40000/40000 [==============================] - 1s 13us/step - loss: 1.0069
Epoch 3/10
40000/40000 [==============================] - 0s 12us/step - loss: 0.7571
Epoch 4/10
40000/40000 [==============================] - 1s 13us/step - loss: 0.6301
Epoch 5/10
40000/40000 [==============================] - 1s 18us/step - loss: 0.5533
Epoch 6/10
40000/40000 [==============================] - 1s 15us/step - loss: 0.5016
Epoch 7/10
40000/40000 [==============================] - 1s 20us/step - loss: 0.4643
Epoch 8/10
40000/40000 [==============================] - 1s 25us/step - loss: 0.4361
Epoch 9/10
40000/40000 [==============================] - 1s 27

40000/40000 [==============================] - 1s 16us/step - loss: 1.0200
Epoch 3/30
40000/40000 [==============================] - 1s 17us/step - loss: 0.7709
Epoch 4/30
40000/40000 [==============================] - 1s 16us/step - loss: 0.6427
Epoch 5/30
40000/40000 [==============================] - 1s 16us/step - loss: 0.5646
Epoch 6/30
40000/40000 [==============================] - 1s 16us/step - loss: 0.5120
Epoch 7/30
40000/40000 [==============================] - 1s 18us/step - loss: 0.4739
Epoch 8/30
40000/40000 [==============================] - 1s 17us/step - loss: 0.4450
Epoch 9/30
40000/40000 [==============================] - 1s 16us/step - loss: 0.4222
Epoch 10/30
40000/40000 [==============================] - 1s 16us/step - loss: 0.4036
Epoch 11/30
40000/40000 [==============================] - 1s 16us/step - loss: 0.3881
Epoch 12/30
40000/40000 [==============================] - 1s 16us/step - loss: 0.3750
Epoch 13/30
40000/40000 [==============================] - 1s 

40000/40000 [==============================] - 1s 15us/step - loss: 0.3312
Epoch 17/50
40000/40000 [==============================] - 1s 15us/step - loss: 0.3244
Epoch 18/50
40000/40000 [==============================] - 1s 15us/step - loss: 0.3182
Epoch 19/50
40000/40000 [==============================] - 1s 14us/step - loss: 0.3126
Epoch 20/50
40000/40000 [==============================] - 1s 16us/step - loss: 0.3075
Epoch 21/50
40000/40000 [==============================] - 1s 16us/step - loss: 0.3027
Epoch 22/50
40000/40000 [==============================] - 1s 14us/step - loss: 0.2982
Epoch 23/50
40000/40000 [==============================] - 1s 14us/step - loss: 0.2940
Epoch 24/50
40000/40000 [==============================] - 1s 15us/step - loss: 0.2901
Epoch 25/50
40000/40000 [==============================] - 1s 16us/step - loss: 0.2865
Epoch 26/50
40000/40000 [==============================] - 1s 14us/step - loss: 0.2830
Epoch 27/50
40000/40000 [==============================

60000/60000 [==============================] - 1s 22us/step - loss: 0.2807
Epoch 11/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.2728
Epoch 12/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.2658
Epoch 13/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.2595
Epoch 14/50
60000/60000 [==============================] - 1s 24us/step - loss: 0.2538
Epoch 15/50
60000/60000 [==============================] - 1s 24us/step - loss: 0.2484
Epoch 16/50
60000/60000 [==============================] - 1s 24us/step - loss: 0.2435
Epoch 17/50
60000/60000 [==============================] - 1s 25us/step - loss: 0.2389
Epoch 18/50
60000/60000 [==============================] - 1s 23us/step - loss: 0.2347
Epoch 19/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.2306
Epoch 20/50
60000/60000 [==============================] - 1s 24us/step - loss: 0.2268
Epoch 21/50
60000/60000 [==============================

Epoch 38/50
54000/54000 [==============================] - 1s 24us/step - loss: 0.1864 - val_loss: 0.1700
Epoch 39/50
54000/54000 [==============================] - 1s 26us/step - loss: 0.1844 - val_loss: 0.1688
Epoch 40/50
54000/54000 [==============================] - 1s 27us/step - loss: 0.1825 - val_loss: 0.1675
Epoch 41/50
54000/54000 [==============================] - 1s 25us/step - loss: 0.1805 - val_loss: 0.1664
Epoch 42/50
54000/54000 [==============================] - 1s 25us/step - loss: 0.1786 - val_loss: 0.1653
Epoch 43/50
54000/54000 [==============================] - 1s 24us/step - loss: 0.1768 - val_loss: 0.1642
Epoch 44/50
54000/54000 [==============================] - 1s 23us/step - loss: 0.1750 - val_loss: 0.1632
Epoch 45/50
54000/54000 [==============================] - 1s 24us/step - loss: 0.1732 - val_loss: 0.1622
Epoch 46/50
54000/54000 [==============================] - 1s 23us/step - loss: 0.1715 - val_loss: 0.1610
Epoch 47/50
54000/54000 [=====================